In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
# df = spark.read.load("/mnt/lsde/datasets/reddit/submissions/RS_2006-05.json.bz2", format="json")

In [0]:
def read_dataset(year_range = [2015, 2016, 2017], month_range = range(1, 13), comments_or_submissions = "comments"):
    if comments_or_submissions is "comments":
        link = "comments/RC_"
    elif comments_or_submissions is "submissions":
        link = "submissions/RS_"
    else:
        link = "non-exist"
    result = None
    for year in year_range:
        for month in month_range:
            month_literal = ""
            if month < 10:
                month_literal = "0" + str(month)
            else:
                month_literal = str(month)
            data_path = "/mnt/lsde/datasets/reddit/" + link + str(year) + "-" + month_literal + ".json.bz2"
            if result is None:
                result = spark.read.load(data_path, format="json")
            else:
                result = result.union(spark.read.load(data_path, format="json"))
    return result

<>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:4: SyntaxWarning: "is" with a literal. Did you mean "=="?
<command-3306133707907135>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
 if comments_or_submissions is "comments":
<command-3306133707907135>:4: SyntaxWarning: "is" with a literal. Did you mean "=="?
 elif comments_or_submissions is "submissions":

In [0]:
import pandas as pd

def monthly_count(year_range = [2015, 2016, 2017], month_range = range(1, 13), comments_or_submissions = "comments"):
    result = None
    for year in year_range:
        for month in month_range:
            df = read_dataset(year_range=[year], month_range=[month])
            if result is None:
                result = pd.DataFrame([[str(year) + "-" + str(month), df.count()]], columns=["date", "count"])
            else:
                result = pd.concat([result, pd.DataFrame([[str(year) + "-" + str(month), df.count()]], columns=["date", "count"])])
    return result

In [0]:
sample_2014 = read_dataset(year_range=[2014], month_range=[4])
sample_2014 = sample_2014.toPandas()
sample_2014.head()
print(sample_2014.shape)

In [0]:
filtered_data = sample_2014[sample_2014["body"].str.contains("ukraine|russia|putin|kyiv|moscow|invasion|vladimir|war")==True]
print(filtered_data.shape)

In [0]:
# # Import seaborn
# import seaborn as sns

# # Apply the default theme
# sns.set_theme()

# # Create a visualization
# sns.barplot(data=graph, x="date", y="count", palette="Blues_d")